In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
#pdf reader

from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs = loader.load()

In [5]:
# converting documents to chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

documents = text_splitter.split_documents(docs)

documents[:5]

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [6]:
#faiss vector DB
#converting text to vectors and storing in vector databases

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

db1 = FAISS.from_documents(documents[:20],OpenAIEmbeddings())

/Users/atul595525/repository1/LLMAPP/llmvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
#searching vectore store using similarity search algorithm

query = "An attention function can be described as mapping a query "

result = db1.similarity_search(query)

result[0].page_content

'3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'

In [8]:
#How to combine prompt with chain with retriever andthen getting getting reponse based on prompt

from langchain_community.llms import Ollama

llm=Ollama(model="llama2")

llm

Ollama()

In [ ]:
#downloading ollama 2 just in case if we don't have it
#open source model
#ollama run llama2
#ollama run llama3

In [9]:
#DESIGN CHAT PROMPT TEMPLATE

from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context} 
</context>
 Question:{input}""")

In [10]:
#CHAINS REFERS TO A sequence of calls whether to an LLM a tool or a data preprocessing step

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [11]:
# Retriever

retriever=db1.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fccf8a94dc0>)

#  Retriever + document chain

Retriever CHAIN: tHIS CHAIN TAKES User inquiry, which is then passed to retreiver to fetch relevant documents. These documents (and original inputs) are then passed to LLM to generate a response

In [12]:
from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [14]:
retrieval_chain.invoke({"input": "An attention function can be described as mapping a query"})

ValueError: Ollama call failed with status code 500. Details: {"error":"error starting the external llama server: fork/exec /var/folders/5n/plkx587s0kj_92g0r6z1vvq40000gn/T/ollama2535701287/runners/metal/ollama_llama_server: no such file or directory "}